#Load the Libraries

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
#Transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    #1st tuple is mean value ans 2nd tuple is SD on rgb channels
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

#Download the datasets

In [4]:
train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:06<00:00, 25.8MB/s]


In [5]:
#dataloader

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

#Build the Architecture

In [9]:
class CNN_Cifar10(nn.Module):
  def __init__(self):
    super(CNN_Cifar10, self).__init__() # Corrected super call

    #feature extractor
    self.cnn1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool1 = nn.MaxPool2d(2,2)
    self.cnn2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)
    self.pool2 = nn.MaxPool2d(2,2)

    #Classifier
    self.flatten = nn.Flatten() # Instantiate Flatten module
    self.fc1 = nn.Linear(6*6*16,4096)
    self.fc2 = nn.Linear(4096, 2048)
    self.fc3 = nn.Linear(2048, 1024)
    self.fc4 = nn.Linear(1024, 512)
    self.fc5 = nn.Linear(512, 256)
    self.fc6 = nn.Linear(256, 10)

  def forward(self, x):
    x = self.cnn1(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.pool1(x)

    x = self.cnn2(x) # Apply cnn2 first
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.pool2(x) # Apply pool2

    x = self.flatten(x) # Use instantiated Flatten

    x = self.fc1(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.fc2(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.fc3(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.fc4(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.fc5(x)
    x = nn.ReLU()(x) # Use instantiated ReLU
    x = self.fc6(x)

    return x

In [10]:
model = CNN_Cifar10()
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#Train

In [ ]:
for epoch in range(10):
  for i, (images, labels) in enumerate(train_loader):
    outputs = model(images)
    loss = criterian(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
_,predicted = torch.max(outputs.data,1)